<a href="https://colab.research.google.com/github/jsqzhu/nbme/blob/main/02_NSMB_Bert_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.backend as K
from tensorflow.keras import mixed_precision
#import tensorflow_addons as tfa
#import dill
import matplotlib.pyplot as plt
import transformers
from transformers import AutoTokenizer, AutoConfig,TFAutoModel
import json
import matplotlib.pyplot as plt
from ast import literal_eval 

In [4]:
train = pd.read_csv('https://raw.githubusercontent.com/jsqzhu/nbme/main/data.csv')
train.head()

,id,case_num,pn_num,feature_num,annotation,location,pn_history,feature_text,annot_count,loc_count
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724'],HPI: 17yo M presents with palpitations. Patien...,Family-history-of-MI-OR-Family-history-of-myoc...,1,1
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693'],HPI: 17yo M presents with palpitations. Patien...,Family-history-of-thyroid-disorder,1,1
2,00016_002,0,16,2,['chest pressure'],['203 217'],HPI: 17yo M presents with palpitations. Patien...,Chest-pressure,1,1
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']",HPI: 17yo M presents with palpitations. Patien...,Intermittent-symptoms,2,2
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258'],HPI: 17yo M presents with palpitations. Patien...,Lightheaded,1,1


## Resources

https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT

# Functions

In [5]:
def extract_location (location):
  """get start_char, end_char from location column"""
  #annotation = literal_eval(annotation)
  location = literal_eval(location)
  #annotation_list = []
  start_char = []
  end_char = []
  #if len(location)==0:
  #  start_char.append(-1)
  #  end_char.append(-1)
  #else:
  for n in range(len(location)):
      loc = location[n].split(' ')
      start_char.append(int(loc[0]))
      end_char.append(int(loc[1]))
  return start_char, end_char

# Prepare Data for Modeling

## Prepare feature entities

In [6]:
print('# unique feature labels: ', train['feature_num'].nunique())

# unique feature labels:  143


In [7]:
train['feature_num'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
       100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 300, 301, 302, 303, 304, 305, 306, 307, 308,
       309, 310, 311, 312, 313, 314, 315, 400, 401, 402, 403, 404, 405,
       406, 407, 408, 409, 500, 501, 502, 503, 504, 505, 506, 507, 508,
       509, 510, 511, 512, 513, 514, 515, 516, 517, 600, 601, 602, 603,
       604, 605, 606, 607, 608, 609, 610, 611, 700, 701, 702, 703, 704,
       705, 706, 707, 708, 800, 801, 802, 803, 804, 805, 806, 807, 808,
       809, 810, 811, 812, 813, 814, 815, 816, 817, 900, 901, 902, 903,
       904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916])

Recall that 4399 cases have no corresponding text in pn_history for the target feature entity. Thus, we need to create an additional class to account for the texts with no annotations

In [8]:
Label = ["O",] + train['feature_num'].unique().tolist() 
le = LabelEncoder()
le.fit(Label)

LabelEncoder()

In [9]:
train['target'] = le.transform(train['feature_num'])

## Prepare dataset

Use transformer with TF2.0: https://www.kdnuggets.com/2021/02/hugging-face-transformer-basics.html#:~:text=NLP%2Dfocused%20startup%20Hugging%20Face,for%20training%20and%20evaluation%20purposes.

In [20]:
# load model
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
test_text = train['pn_history'][0]

https://huggingface.co/docs/transformers/main_classes/tokenizer

In [22]:
encodings = tokenizer(test_text, 
                           max_length=512,
                           padding='max_length',#sentence max to save on computation
                           truncation=True,
                           return_offsets_mapping=True)

In [23]:
print(f'text: \'{test_text}\'')
print(f'input ids: {encodings["input_ids"]}')
print(f'attention mask: {encodings["attention_mask"]}')
print(f'offset_mapping: {encodings["offset_mapping"]}')

text: 'HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of "heart beating/pounding out of my chest." 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. 
PMHx: none
Rx: uses friends adderrall
FHx: mom with "thyroid disease," dad with recent heart attcak
All: none
Immunizations: up to date
SHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms'
input ids: [101, 6857, 1182, 131, 1542, 7490, 182, 8218, 11

In [24]:
for token_id, pos in zip(encodings['input_ids'], encodings['offset_mapping']):
    print(token_id, pos, test_text[pos[0]:pos[1]])

101 (0, 0) 
6857 (0, 2) HP
1182 (2, 3) I
131 (3, 4) :
1542 (5, 7) 17
7490 (7, 9) yo
182 (10, 11) M
8218 (12, 20) presents
1114 (21, 25) with
185 (26, 27) p
1348 (27, 29) al
18965 (29, 32) pit
6006 (32, 38) ations
119 (38, 39) .
5351 (40, 47) Patient
3756 (48, 55) reports
124 (56, 57) 3
118 (57, 58) -
125 (58, 59) 4
1808 (60, 66) months
1104 (67, 69) of
27946 (70, 82) intermittent
3426 (83, 91) episodes
1104 (92, 94) of
107 (95, 96) "
1762 (96, 101) heart
5405 (102, 109) beating
120 (109, 110) /
9683 (110, 118) pounding
1149 (119, 122) out
1104 (123, 125) of
1139 (126, 128) my
2229 (129, 134) chest
119 (134, 135) .
107 (135, 136) "
123 (137, 138) 2
1552 (139, 143) days
2403 (144, 147) ago
1219 (148, 154) during
170 (155, 156) a
5862 (157, 163) soccer
1342 (164, 168) game
1125 (169, 172) had
1126 (173, 175) an
2004 (176, 183) episode
117 (183, 184) ,
1133 (185, 188) but
1142 (189, 193) this
1159 (194, 198) time
1125 (199, 202) had
2229 (203, 208) chest
2997 (209, 217) pressure
1105 (218,

In [25]:
# need to check the max length
# check if len(subtoken) longer than max_length from the model?
#https://ai.stackexchange.com/questions/37624/why-do-transformers-have-a-fixed-input-length
# how to handle longer tokens
#https://stackoverflow.com/questions/74290497/how-to-handle-sequences-longer-than-512-tokens-in-layoutlmv3
# for offset_mapping
# https://huggingface.co/transformers/v4.2.2/custom_datasets.html

use model tokenizer to prepare input text; 
convert pn_history to word embedding as input features 

In [26]:
train_copy = train.copy()

In [27]:
train_copy[train_copy['location'].str.contains(';')].head()

,id,case_num,pn_num,feature_num,annotation,location,pn_history,feature_text,annot_count,loc_count,target
48,00082_009,0,82,9,"['heart pounding', 'heart racing', 'heart poun...","['85 99', '126 138', '126 131;143 151']",17 yo M w/ no cardiac or arrhythmia PMH presen...,heart-pounding-OR-heart-racing,3,3,125
91,00211_000,0,211,0,['Father MI'],['682 688;695 697'],HPI: Patient is a 17 yo m with a c/o of palpit...,Family-history-of-MI-OR-Family-history-of-myoc...,1,1,0
96,00211_005,0,211,5,['Denies sweating'],['462 468;473 481'],HPI: Patient is a 17 yo m with a c/o of palpit...,No-hair-changes-OR-no-nail-changes-OR-no-tempe...,1,1,64
100,00211_009,0,211,9,"['palpitations', 'Palpitations', 'palpitations...","['40 52', '55 67', '104 116', '161 178', '161 ...",HPI: Patient is a 17 yo m with a c/o of palpit...,heart-pounding-OR-heart-racing,5,5,125
125,00225_008,0,225,8,"['drinks coffee', 'drinks energy drinks']","['461 467;483 489', '461 467;506 519']",17 y/o previously healthy male here with heart...,Caffeine-use,2,2,106


In [28]:
# remove records with error in annotation/location; will investigate later
train = train[~train['location'].str.contains(';')]

In [29]:
train['start_char'], train['end_char'] = zip(*train['location'].apply(lambda x: extract_location(x)))

In [30]:
train = train.explode(['start_char','end_char'])

In [31]:
train.head()

,id,case_num,pn_num,feature_num,annotation,location,pn_history,feature_text,annot_count,loc_count,target,start_char,end_char
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724'],HPI: 17yo M presents with palpitations. Patien...,Family-history-of-MI-OR-Family-history-of-myoc...,1,1,0,696,724
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693'],HPI: 17yo M presents with palpitations. Patien...,Family-history-of-thyroid-disorder,1,1,1,668,693
2,00016_002,0,16,2,['chest pressure'],['203 217'],HPI: 17yo M presents with palpitations. Patien...,Chest-pressure,1,1,18,203,217
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']",HPI: 17yo M presents with palpitations. Patien...,Intermittent-symptoms,2,2,36,70,91
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']",HPI: 17yo M presents with palpitations. Patien...,Intermittent-symptoms,2,2,36,176,183


## SEQUENCE Length

In [73]:
pn_history=[]
input_id_sequence = []
attn_sequence = []
Target_labels = []
max_sequence_length = 0
for pn in train.groupby('pn_num'):
  # for loop to get encoding for each unique clinical note
  # output is a tuple (pn_num, df)
  df = pn[1] 
  #clinical note to be annotated
  pn_history = df.iloc[0].pn_history
  
  # get encoding
  encoding = tokenizer(pn_history, 
                       #max_length=512,
                       padding='max_length',#sentence max to save on computation
                       truncation=False,
                       return_offsets_mapping=True)
  input_id = encoding['input_ids']
  attn = encoding['attention_mask']
  offset = encoding['offset_mapping']

  sequence_length = len(input_id)
  if sequence_length > max_sequence_length:
    max_sequence_length = sequence_length
  else:
    pass

print('max_sequence_length: ', max_sequence_length)

max_sequence_length:  300


## Testing target labeling

In [32]:
test = train[train.pn_num==16]

In [33]:
test

,id,case_num,pn_num,feature_num,annotation,location,pn_history,feature_text,annot_count,loc_count,target,start_char,end_char
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724'],HPI: 17yo M presents with palpitations. Patien...,Family-history-of-MI-OR-Family-history-of-myoc...,1,1,0,696,724
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693'],HPI: 17yo M presents with palpitations. Patien...,Family-history-of-thyroid-disorder,1,1,1,668,693
2,00016_002,0,16,2,['chest pressure'],['203 217'],HPI: 17yo M presents with palpitations. Patien...,Chest-pressure,1,1,18,203,217
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']",HPI: 17yo M presents with palpitations. Patien...,Intermittent-symptoms,2,2,36,70,91
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']",HPI: 17yo M presents with palpitations. Patien...,Intermittent-symptoms,2,2,36,176,183
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258'],HPI: 17yo M presents with palpitations. Patien...,Lightheaded,1,1,53,222,258
5,00016_005,0,16,5,[],[],HPI: 17yo M presents with palpitations. Patien...,No-hair-changes-OR-no-nail-changes-OR-no-tempe...,0,0,64,NaN,NaN
6,00016_006,0,16,6,"['adderall', 'adderrall', 'adderrall']","['321 329', '404 413', '652 661']",HPI: 17yo M presents with palpitations. Patien...,Adderall-use,3,3,83,321,329
6,00016_006,0,16,6,"['adderall', 'adderrall', 'adderrall']","['321 329', '404 413', '652 661']",HPI: 17yo M presents with palpitations. Patien...,Adderall-use,3,3,83,404,413
6,00016_006,0,16,6,"['adderall', 'adderrall', 'adderrall']","['321 329', '404 413', '652 661']",HPI: 17yo M presents with palpitations. Patien...,Adderall-use,3,3,83,652,661


https://www.lighttag.io/blog/sequence-labeling-with-transformers/example

In [34]:
# create a dictionary of target and start_char, and target and end_char
#target_start_dict = {k:[j for i in v for j in i] for k, v in test.groupby('target')['start_char']}
#target_end_dict = {k:[j for i in v for j in i] for k, v in test.groupby('target')['end_char']}

In [35]:
# tokenize the text 
encoding = tokenizer(test['pn_history'][0], 
                       #max_length='512',
                       padding='longest',#sentence max to save on computation
                      truncation=False,
                        return_offsets_mapping=True)
input_id = encoding['input_ids']
attn = encoding['attention_mask']
offset = encoding['offset_mapping']

In [36]:
len(offset)

253

In [37]:
# create a list same length as tokens to store target labels
Target_entity_labels = []
Target_entity_labels = ["O"]*len(offset)
# iterate through rows in dataframe to compare start/end char with the offset mapping position
for idx, row in test.iterrows(): 
  # compare
  start = row.start_char
  end = row.end_char
  target = row.target
  for i, (pos_1, pos_2) in enumerate(offset):
    if (pos_1 >= start) and (pos_2 <= end): #
      Target_entity_labels[i] = target
    else:
      pass
#print(Target_entity_labels)

In [38]:
Target_entity_labels

['O',
 'O',
 'O',
 'O',
 13,
 13,
 17,
 'O',
 'O',
 125,
 125,
 125,
 125,
 'O',
 'O',
 'O',
 2,
 2,
 2,
 2,
 2,
 36,
 36,
 'O',
 'O',
 125,
 125,
 125,
 125,
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 36,
 'O',
 'O',
 'O',
 'O',
 'O',
 18,
 18,
 'O',
 53,
 53,
 53,
 53,
 53,
 53,
 53,
 53,
 53,
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 83,
 83,
 83,
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 83,
 83,
 83,
 83,
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 83,
 83,
 83,
 83,


## Prepare data for Bert

In [74]:
pn_history=[]
input_id_sequence = []
attn_sequence = []
Target_labels = []
for pn in train.groupby('pn_num'):
  # for loop to get encoding for each unique clinical note
  # output is a tuple (pn_num, df)
  df = pn[1] 
  #clinical note to be annotated
  pn_history = df.iloc[0].pn_history
  
  # get encoding
  encoding = tokenizer(pn_history, 
                       max_length=max_sequence_length,
                       padding='max_length',#sentence max to save on computation
                       truncation=False,
                       return_offsets_mapping=True)
  input_id = encoding['input_ids']
  attn = encoding['attention_mask']
  offset = encoding['offset_mapping']

  # prepare target sequence
  # First, create a target_entity_sequence same length as encoding and fill by default 'O'
  Target_label = ["O"]*len(offset) 
  # Next, iterate through rows in df to compare start/end char with the offset_mapping position
  for idx, row in df.iterrows(): 
    start = row.start_char
    end = row.end_char
    target = row.target

    for i, (pos_1, pos_2) in enumerate(offset):
      if (pos_1 >= start) and (pos_2 <= end): #
        Target_label[i] = target
      else:
        pass
  Target_labels.append(Target_label)
  input_id_sequence.append(input_id)
  attn_sequence.append(attn)


In [75]:
print(len(input_id_sequence))
print(len(attn_sequence))
print(len(Target_labels))

1000
1000
1000


In [76]:
print(len(input_id_sequence[0]))
print(len(attn_sequence[0]))
print(len(Target_labels[0]))

300
300
300


In [77]:
pn_history

'Stephanie madden is a 20 year old woman complaining of headache. Pain began yesterday morning and has been getting worse. Dull constant ache bilaterally, worse with walking, bending over, no alleviating factors. Did not respond to ibuprofen, Tylenol, or sleep. Complains of photobia, but not phonophobia, no aura. Subjective fever today. Neck stiffness. Nausea and vomiting. No dizzyness or lightheadedness, no weakness, no sick contacts.\r\nROS: otherwise negative\r\nPMH: none\r\nPSH: none\r\nHosp: none\r\nMed: Birth control pill\r\nAllergies: none\r\nSoc: works at sporting good store, lives with roomate, no tobacoo, etoh 2-3 drinks on weekends, marijuna 3-4 joints a week, sexually active and uses condoms'

In [78]:
np.array(input_id)

array([  101,  2585, 22808,  1162,  6340,  2883,  1110,   170,  1406,
        1214,  1385,  1590, 19533,  1104, 16320,   119,  2489,  1310,
        8128,  2106,  1105,  1144,  1151,  2033,  4146,   119, 10884,
        4836, 12953, 20557,  1193,   117,  4146,  1114,  3179,   117,
       16571,  1166,   117,  1185,  1155,  6348, 25148,  5320,   119,
        1225,  1136,  6297,  1106,   178,  7925,  1643,  2180, 13488,
         117,   189, 12415,  2728,  1233,   117,  1137,  2946,   119,
       19073,  1116,  1104,  6307, 10242,   117,  1133,  1136,   185,
        8613,  4184,  5114, 10242,   117,  1185, 19035,   119, 23481,
       10880,  2052,   119,  2455, 11111,  1757,   119, 22882,  1105,
       26979,  1158,   119,  1185, 21441,  1757,  1137,  1609, 14443,
        1757,   117,  1185, 11477,   117,  1185,  4809, 10492,   119,
         187,  2155,   131,  4303,  4366,  9852,  1324,   131,  3839,
       15604,  1324,   131,  3839, 16358, 20080,   131,  3839,  1143,
        1181,   131,

In [79]:
np.array(attn)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [81]:
len(attn)

300